# Download SOILGRIDS-derived soil texture class map
SOILGRIDS data (Hengl et al., 2017) showing global sand, silt and clay percentages have been downloaded and processed into a global map of USDA soil texture classes. See: https://www.hydroshare.org/resource/1361509511e44adfba814f6950c6e742/

This script downloads and unpacks this global map.


In [1]:
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime
from hs_restclient import HydroShare, HydroShareAuthBasic

#### Control file handling

In [2]:
# Easy access to control file folder
controlFolder = Path('../../../0_control_files')

In [3]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [4]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line:
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [5]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find the Hydroshare download ID

In [6]:
# Find the ID in the control file
download_ID = read_from_control(controlFolder/controlFile,'parameter_soil_hydro_ID')

#### Find where to save the data

In [7]:
# Find the path where the raw soil maps need to go
soil_path = read_from_control(controlFolder/controlFile,'parameter_soil_raw_path')

In [8]:
# Specify the default paths if required 
if soil_path == 'default':
    soil_path = make_default_path('parameters/soilclass/1_soil_classes_global') # outputs a Path()
else:
    soil_path = Path(soil_path) # make sure a user-specified path is a Path()

In [9]:
# Make the folder if it doesn't exist
soil_path.mkdir(parents=True, exist_ok=True)

#### Get authentication info

In [10]:
# Open the login details file and store as a dictionary
hydroshare_login = {}
with open(os.path.expanduser("~/.hydroshare")) as file:
    for line in file:
        (key, val) = line.split(':')
        hydroshare_login[key] = val.strip() # remove whitespace, newlines

In [11]:
# Get the authentication details
usr = hydroshare_login['name']
pwd = hydroshare_login['pass']

#### Download the data

In [12]:
# Authenticate the user
auth = HydroShareAuthBasic(username = usr, password = pwd)

In [13]:
# Make a hydroshare object - note: needs authentication
hs = HydroShare(auth=auth)

In [18]:
# Specify the resource ID and download the resource data
#out = hs.getResource(download_ID, destination=soil_path)
out = hs.getResourceFile(download_ID, "usda_mode_soilclass_250m_ll.tif", destination = soil_path)

In [20]:
# Check for output messages
print(out)

C:\Globus endpoint\summaWorkflow_data\domain_BowAtBanff\parameters\soilclass\1_soil_classes_global\usda_mode_soilclass_250m_ll.tif


#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [21]:
# Set the log path and file name
logPath = soil_path
log_suffix = '_soilclass_download_log.txt'

In [22]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [23]:
# Copy this script
thisFile = 'download_soilclass_global_map.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [24]:
# Get current date and time
now = datetime.now()

In [25]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Downloaded SOILGRIDS-derived soil texture classes (global coverage).']
    for txt in lines:
        file.write(txt)  